<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/C4_FAQs_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -Uq transformers sentence-transformers datasets sentencepiece

In [59]:
from datasets import load_dataset
from tqdm.auto import tqdm
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize # We need a fast sentence-tokenizer
from typing import List
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
dataset = load_dataset("c4", "en", split="train", streaming=True)

In [80]:
def tokenize_text(text: str):
  sentence_tokens = sent_tokenize(text)
  return sentence_tokens

def num_word_tokens(text: str):
  return len(word_tokenize(text))

def flow(sentence_tokens: List[str]):
  initial_question_found = False
  questions, answers = [], []
  for i, sent in enumerate(sentence_tokens):
    if sent.endswith("?"):
      questions.append(sent)
      answer_j = []
      for sent_j in sentence_tokens[i+1:]:
        if not sent_j.endswith("?"):
          answer_j.append(sent_j)
        else:
          break
      answer_j_text = " ".join(answer_j)
      answers.append(answer_j_text)
  
  # print(f"Total questions: {len(questions)}")
  # print(f"Total answers: {len(answers)}")
  qa_pairs = list(zip(questions, answers))
  qa_pairs = list(filter(lambda x: num_word_tokens(x[0]) > 2 and \
                         num_word_tokens(x[1]) > 4, qa_pairs))
  return qa_pairs

In [81]:
count_faqs = 0
done_urls = set()
limit = 5000
progress_bar = tqdm(range(limit))
all_questions, all_answers = [], []
for dict_ in dataset:
  if count_faqs == limit:
    break

  url = dict_["url"]
  text = dict_["text"]
  if "faq" in url.lower():
    count_faqs += 1
    progress_bar.update(1)
    done_urls.add(url)
    qa_pairs = flow(tokenize_text(text))
    if qa_pairs:
      questions, answers = zip(*qa_pairs)
      questions, answers = list(questions), list(answers)
      all_questions.append(questions), all_answers.append(answers)

  0%|          | 0/5000 [00:00<?, ?it/s]

In [82]:
print(f"Total filtered faq pages: {len(all_questions)}")

Total filtered faq pages: 3804


In [93]:
all_questions[2004]

['Are there any medical tests required before enrolling for IVF program?',
 'How to choose a Surrogate Mother?',
 'We want to use our own eggs and sperms and want to use a surrogate only, it is possible?',
 'How long does it take on average from beginning (signing contract) till the time you take baby home?',
 'What is the process for arranging a Caucasian egg donor- and does this cost very much?',
 'Whose names come on baby birth Certificate?',
 'It is possible to do shipment of embryos/sperms?',
 'What are some of the benefits of having a Client Manager like you?']

In [94]:
all_answers[2004]

['For female partner: Hormone tests like FSH, LH, Estradiol, Prolactin, TSH and AMH. These tests can be performed on Day2 to Day3 of her cycle and pelvic ultrasound to rule out possibility of fibroids, cysts, polyps, or fluid in the uterus. For both partners: There are few infectious disease tests which are mandatory for both partners. These tests are done to make sure that prospective parents are not passing any disease to their offspring. These tests include hepatitis B and C, HIV, Gonorrhea and Chlamydia, syphilis. There might be few additional tests as per the screening guidelines for that specific country. In case if you are using egg donor/sperm donor, medical tests is not required for partner who is not donating his/her biological materials. These tests are valid for 3-6 months.',
 'Generally surrogate mother will be recruited for your program by your medical team based on her previous obstetrical history and medical tests. It is important to choose a surrogate mother who is at 

In [88]:
sum([len(x) for x in all_questions])

23571